# 🎙️ Unified Podcast Generator - Google Colab

**Full-featured AI Podcast Generator**
- 📄 Document upload (PDF, DOCX, TXT)
- 🤖 LLM script generation (Mistral/Llama)
- 🎤 Voice cloning (XTTS v2 via coqui-tts)
- 🎛️ Advanced audio settings

## ⚡ Setup: Enable GPU → `Runtime` → `Change runtime type` → `T4 GPU`

## Step 1: Clone & Install

In [ ]:
# Clone repository
!git clone https://github.com/beginner4a3/pod2.git
%cd pod2

# Set HuggingFace token (REPLACE with your token)
import os
os.environ['HF_TOKEN'] = "your_token_here"  # Get from: https://huggingface.co/settings/tokens

# Install ALL dependencies
!pip install -r requirements.txt -q

print("\n✅ All dependencies installed!")

## Step 2: Download Models

In [ ]:
# Download TTS model
from huggingface_hub import snapshot_download
import os

print("📥 Downloading Indic-ParlerTTS model (~3GB)...")
snapshot_download(repo_id="ai4bharat/indic-parler-tts", token=os.environ.get('HF_TOKEN'))
print("✅ TTS Model ready!")

# Download LLM model
print("\n📥 Downloading Mistral-7B LLM (~4GB)...")
!wget -q --show-progress https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf
print("✅ LLM Model ready!")

## Step 3: Verify Installation

In [ ]:
import sys
sys.path.insert(0, '/content/pod2')

from src.tts.indic_parler import IndicParlerTTS
from src.tts.xtts_cloner import is_voice_cloning_available

print("✅ Indic-ParlerTTS: Ready")
print(f"✅ Voice Cloning (XTTS v2): {'Available' if is_voice_cloning_available() else 'Not installed'}")
print(f"✅ Languages: {len(IndicParlerTTS.get_languages())} supported")

## Step 4: Launch UI 🚀

In [ ]:
from src.ui.gradio_app import create_interface

print("🚀 Starting Podcast Generator...")
print("💡 LLM Model Path: mistral-7b-instruct-v0.2.Q4_K_M.gguf\n")

demo = create_interface()
demo.queue().launch(share=True)

---

## Alternative: Manual Usage

In [ ]:
# Quick TTS test
from src.tts.indic_parler import IndicParlerTTS
from IPython.display import Audio

tts = IndicParlerTTS()
audio = tts.generate("नमस्ते! यह एक परीक्षण है।", speaker="Rohit", emotion="happy")
tts.save(audio, "test.wav")
Audio("test.wav")

In [ ]:
# Voice cloning - upload reference audio
from google.colab import files
from src.tts.xtts_cloner import XTTSCloner

print("Upload a ~6 second voice sample:")
uploaded = files.upload()
ref_audio = list(uploaded.keys())[0]

cloner = XTTSCloner()
cloned = cloner.generate("Hello, this is my cloned voice!", ref_audio, "en")

import soundfile as sf
sf.write("cloned.wav", cloned, 24000)
Audio("cloned.wav")

In [ ]:
# Generate podcast
from src.tts.indic_parler import IndicParlerTTS
from src.audio.mixer import PodcastMixer, AudioClip
from IPython.display import Audio

script = """Speaker1: नमस्ते दोस्तों! आज हम AI के बारे में बात करेंगे।
Speaker2: हाँ, यह बहुत interesting topic है।"""

tts = IndicParlerTTS()
clips = []

for line in script.strip().split('\n'):
    if ':' in line:
        speaker, text = line.split(':', 1)
        speaker_name = "Rohit" if "1" in speaker else "Divya"
        audio = tts.generate(text.strip(), speaker=speaker_name, emotion="conversation")
        clips.append(AudioClip(audio=audio, sample_rate=tts.sample_rate, speaker=speaker_name))

mixer = PodcastMixer(sample_rate=tts.sample_rate)
final = mixer.mix_turns(clips, gap_ms=100, add_noise=True)

import soundfile as sf
sf.write("podcast.wav", final, tts.sample_rate)
Audio("podcast.wav")

In [ ]:
# Download audio
from google.colab import files
files.download("podcast.wav")